## Preprocess data

In [ ]:
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, get_linear_schedule_with_warmup
from torch.optim import AdamW
import torch
import pytorch_lightning as pl
from tqdm import tnrange
from collections import Counter


# from evaluate import load
# import bert_score
from transformers import logging
logging.set_verbosity_error()
import matplotlib.pyplot as plt
# import bert_score


from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor


import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
dataset = load_dataset("xsum")

use_percent_train = 15
dataset_train = load_dataset("xsum", split=f"train[:{use_percent_train}%]")
dataset_val = load_dataset("xsum", split=f"validation[:{2*use_percent_train}%]")
dataset_test = load_dataset("xsum", split=f"test[:{2*use_percent_train}%]")
dataset = DatasetDict({'train': dataset_train, 'validation': dataset_val, 'test': dataset_test})

print(dataset)

In [ ]:
example = dataset['train'][0]

print("text:", example["document"])
print("Summary:", example["summary"])
print("id:", example["id"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prefix = "summarize: "
max_input_length = 600
max_target_length = 200

def preprocessor(examples, prefix='summarize:', max_input_length=512, max_target_length=128):
  # encode the code-docstring pairs
  texts = examples['document']
  summaries = examples['summary']
  
  inputs = [prefix + text for text in texts]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(summaries, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

In [ ]:
dataset = dataset.map(preprocessor, batched=True)
print(dataset)

In [ ]:
dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])

print(dataset)

In [ ]:
num_workers = os.cpu_count()
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8, num_workers=num_workers)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4, num_workers=num_workers)
test_dataloader = DataLoader(dataset['test'], batch_size=4, num_workers=num_workers)

In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())

In [ ]:
tokenizer.decode(batch['input_ids'][0])

In [ ]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

## Fine-tune using PyTorch Lightning


In [ ]:
class T5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=15, warmup_steps=1000):
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
        self.train_losses=[]
        self.validation_losses=[]
        self.config=self.model.config

        self.train_losses_epoch=[]
        self.validation_losses_epoch=[]

        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)
        self.train_losses.append(loss)
        

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)
        self.validation_losses.append(loss)
        
            
        return loss

    
        
    
        return loss
    def on_train_epoch_end(self):
        # Calculate average loss for the epoch and append to the list
        avg_train_loss = sum(self.train_losses)/ len(self.train_losses)
        self.train_losses_epoch.append(avg_train_loss.item())

        # Reset epoch loss accumulator
        self.train_losses = []

    def on_validation_epoch_end(self):
        # Calculate average loss for the epoch and append to the list
        avg_val_loss = sum(self.validation_losses) / len(self.validation_losses)
        self.validation_losses_epoch.append(avg_val_loss.item())

        # Reset epoch loss accumulator
        self.validation_losses = []

    

        # Reset epoch loss accumulator
        self.test_losses = []
    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.model.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
    
    def generate(self, input_ids, max_new_tokens=100, device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
        input_ids=input_ids.clone().detach().reshape((1,-1)).to(device)
        return self.model.generate(input_ids, max_new_tokens=max_new_tokens)
    
    def push_to_hub(self, model_name, organization):
        # Save the model
        self.model.push_to_hub(model_name, organization)

        
    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
# Hyperparameters
lr = 5e-5
num_train_epochs = 10
warmup_steps = 1000
patience = 2
max_epochs = 10

# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=patience,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')
checkpoint_callback = ModelCheckpoint(dirpath='./', monitor='validation_loss', mode='min', save_top_k = 1)

accelerator = "gpu" if torch.cuda.is_available() else "cpu"
trainer = Trainer(accelerator=accelerator, 
                  callbacks=[early_stop_callback, lr_monitor, checkpoint_callback], max_epochs=max_epochs)

model = T5(lr=lr, num_train_epochs=num_train_epochs, warmup_steps=warmup_steps)
trainer.fit(model)
tokenizer.save_pretrained("./")
checkpoint_path = checkpoint_callback.best_model_path

In [ ]:
train_losses=model.train_losses_epoch
validation_losses=model.validation_losses_epoch


In [ ]:
print('Loss on validation set before fine tuning: ', validation_losses[0])

In [ ]:

# Plotting the losses
plt.plot(train_losses, label='Training Loss')
plt.plot(validation_losses[1:], label='Validation Loss')

# Adding labels and title
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('T5: Losses over Epochs')

# Adding legend
plt.legend()

plt.savefig('losses_plot.png')
# Displaying the plot
plt.show()

### Push to hub

In [ ]:
# import json
# model_file='.t5_model.bin'
# config_file='.t5_config.json'

# torch.save(model.state_dict(), model_file)
# model.config.to_json_file(config_file)

# from transformers import T5Config, AutoConfig, AutoModel, T5ForConditionalGeneration

# config = T5Config.from_json_file(config_file)
# test_model = T5ForConditionalGeneration(config)
# state_dict = torch.load(model_file)
# new_state_dict = {key.replace("model.", ""): value for key, value in state_dict.items()}
# test_model.load_state_dict(new_state_dict)
# test_model.eval()

## Inference

In [ ]:
data_id = 9
test_dataset = dataset['test']
test_document = test_dataset['document'][data_id]
test_input_ids = test_dataset[data_id]['input_ids']
test_gt_summary = test_dataset['summary'][data_id]
test_attention_mask = test_dataset[data_id]['attention_mask']

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

In [ ]:
print("Input document:")
print(test_document)

In [ ]:
print("Ground truth summary:")
print(test_gt_summary)

In [ ]:
output_ids = model.generate(test_input_ids.reshape((1,-1)))
generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
print("Generated summary:")
print(generated_summary)

### append the results

In [ ]:
# def append_results(article, ground_truth_summary, generated_summary, filename="results.txt"):
#     # Initialize the result string in a readable format
#     result = f"\n\n---\n\nArticle:\n{article}\n\nGround Truth Summary:\n{ground_truth_summary}\n\nGenerated Summary:\n{generated_summary}\n\n---\n"
    
#     # Open the file in append mode, if the file does not exist it will be created
#     with open(filename, "a", encoding="utf-8") as file:
#         file.write(result)

#     print(f'Results have been appended successfully to {filename}')


# append_results(test_document, test_gt_summary, generated_summary)

### Save the files

In [ ]:
# import os
# import zipfile

# def zip_files(*file_names, zip_name="archive.zip"):
#     # Make sure all file names are strings and the files exist
#     for name in file_names:
#         if not isinstance(name, str) or not os.path.isfile(name):
#             raise ValueError(f"{name} is not a valid file name.")

#     # Create a new zip file (or overwrite an existing one)
#     with zipfile.ZipFile(zip_name, 'w') as zipf:
#         for name in file_names:
#             zipf.write(name)

#     print(f'Files have been zipped successfully into {zip_name}')

# zip_files("special_tokens_map.json", "spiece.model", "losses_plot.png", "tokenizer_config.json", "tokenizer.json")

### Push to Huggingface

In [ ]:
# from transformers import T5Config, T5ForConditionalGeneration

# model_file='./t5_model.bin'
# config_file='./t5_config.json'

# torch.save(model.state_dict(), model_file)
# model.config.to_json_file(config_file)

# config = T5Config.from_json_file(config_file)
# test_model = T5ForConditionalGeneration(config)
# state_dict = torch.load(model_file)
# new_state_dict = {key.replace("model.", ""): value for key, value in state_dict.items()}
# test_model.load_state_dict(new_state_dict)
# test_model.eval()

# from huggingface_hub import notebook_login
# notebook_login()
# test_model.push_to_hub("T5_summarizer", organization="ZinebSN")
# tokenizer.push_to_hub("T5_summarizer", organization="ZinebSN")

# tokenizer.save_pretrained("ZinebSN/T5_summarizer")